<a href="https://colab.research.google.com/github/m-r-tanha/Predict-Time-Series-Trend-with-RNN-recurrent-Neural-Network-/blob/master/RNN_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, normalize, minmax_scale
from tensorflow.contrib import rnn
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
from google.colab import files
import io
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, RNN, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file


tbc=TensorBoardColab()

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Wait for 8 seconds...
TensorBoard link:
http://bd44766b.ngrok.io


Saving Payload_4year_trend.xlsx to Payload_4year_trend.xlsx
User uploaded file "Payload_4year_trend.xlsx" with length 265443 bytes


In [0]:
from keras import backend as K
import tensorflow as tf
from sklearn.metrics import roc_curve

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

tf.compat.v1.get_default_graph()

df = pd.read_excel("Payload_4year_trend.xlsx",sheet_name='Payload')
test_set = pd.read_excel("Payload_4year_trend.xlsx",sheet_name='Test')

real_test = test_set.iloc[:,1:2].values
input_data= df.iloc[:,1:2].values
sc = MinMaxScaler()
input_data = sc.fit_transform(input_data)
X_train = input_data[0:1493]
Y_train = input_data[7:1500]

X_test = real_test[0:169]
Y_test= real_test[7:176]

X_train = np.reshape(X_train,(1493,1,1))
X_test = np.reshape(X_test,(169,1,1))

#activation_function=['relu']
#activation_function=['tanh']
#Optimizer_=[ 'Adagrad']
#activation_function=['sigmoid','relu','tanh','softmax']
#activation_function=['sigmoid']
#activation_function=['tanh','sigmoid']
#Optimizer_=[ 'Adam','sgd']
#Optimizer_=['sgd','RMSprop','Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

RMSE=['Activation function','optimizer','RMSE']
evalution=pd.DataFrame([['Activation function','optimizer','ERROR','loss', 'accuracy', 'f1_score', 'precision', 'recall']])

for j in Optimizer_:
    for i in activation_function:

        RNN_Model = Sequential()
        RNN_Model.add(GRU(units=4,activation=i,input_shape=(None,1)))
        RNN_Model.add(Dropout(0.1))
        RNN_Model.add(Dense(units=1))
        RNN_Model.compile(optimizer = j, loss = 'mean_squared_error', metrics=['accuracy', f1_m,precision_m, recall_m])
        History=RNN_Model.fit(X_train, Y_train, batch_size = 15, epochs = 200) 
        loss, accuracy, f1_score, precision, recall = RNN_Model.evaluate(X_test, Y_test, verbose=0)
        epoch=199
        #print(History.history)
        inputs = real_test
        inputs = sc.transform(inputs)
        inputs = np.reshape(inputs, (176, 1, 1))
        predicted = RNN_Model.predict(inputs)
        predicted = sc.inverse_transform(predicted)

        plt.plot(real_test, color = 'red', label = 'Real ')
        plt.plot(predicted, color = 'blue', label = 'Predicted ')
        title_='Activation function= '+i+'   optimizer='+j
        plt.title(title_)
        plt.xlabel('Time')
        plt.ylabel('GByte')
        plt.legend()
        fig = plt.gcf()
        fig.set_size_inches(10, 6)
        plt.show()
        rmse = math.sqrt(mean_squared_error(real_test, predicted))
        RMSE.append([i,j,rmse])
        print ("Error: "+str(rmse))
        evalu=pd.DataFrame([[i, j, rmse, loss, accuracy, f1_score, precision, recall]])
        evalution=evalution.append(evalu)
print(evalution)


In [26]:
cm = confusion_matrix(Y_test, Y_pred)

1.0